In [1]:
%run init.ipynb

matchzoo version 1.0
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`


In [2]:
preprocessor = mz.models.Bert.get_default_preprocessor()

In [3]:
train_pack_processed = preprocessor.transform(train_pack_raw)
dev_pack_processed = preprocessor.transform(dev_pack_raw)
test_pack_processed = preprocessor.transform(test_pack_raw)

Processing length_right with len: 100%|██████████| 2300/2300 [00:00<00:00, 569743.63it/s]


In [4]:
trainset = mz.dataloader.Dataset(
    data_pack=train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1
)
testset = mz.dataloader.Dataset(
    data_pack=test_pack_processed
)

In [5]:
padding_callback = mz.models.Bert.get_default_padding_callback()
trainloader = mz.dataloader.DataLoader(
    dataset=trainset,
    batch_size=20,
    stage='train',
    resample=True,
    sort=False,
    callback=padding_callback
)
testloader = mz.dataloader.DataLoader(
    dataset=testset,
    batch_size=20,
    stage='dev',
    callback=padding_callback
)

In [6]:
model = mz.models.Bert()

model.params['task'] = ranking_task
model.params['mode'] = 'bert-base-uncased'
model.params['dropout_rate'] = 0.2

model.build()

print(model)
print('Trainable params: ', sum(p.numel() for p in model.parameters() if p.requires_grad))

Bert(
  (bert): BertModule(
    (bert): BertModel(
      (embeddings): BertEmbeddings(
        (word_embeddings): Embedding(30522, 768, padding_idx=0)
        (position_embeddings): Embedding(512, 768)
        (token_type_embeddings): Embedding(2, 768)
        (LayerNorm): BertLayerNorm()
        (dropout): Dropout(p=0.1, inplace=False)
      )
      (encoder): BertEncoder(
        (layer): ModuleList(
          (0): BertLayer(
            (attention): BertAttention(
              (self): BertSelfAttention(
                (query): Linear(in_features=768, out_features=768, bias=True)
                (key): Linear(in_features=768, out_features=768, bias=True)
                (value): Linear(in_features=768, out_features=768, bias=True)
                (dropout): Dropout(p=0.1, inplace=False)
              )
              (output): BertSelfOutput(
                (dense): Linear(in_features=768, out_features=768, bias=True)
                (LayerNorm): BertLayerNorm()
                (dr

In [7]:
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 5e-5},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

from pytorch_transformers import AdamW, WarmupLinearSchedule

optimizer = AdamW(optimizer_grouped_parameters, lr=5e-5, betas=(0.9, 0.98), eps=1e-8)
scheduler = WarmupLinearSchedule(optimizer, warmup_steps=6, t_total=-1)

trainer = mz.trainers.Trainer(
    model=model,
    optimizer=optimizer,
    scheduler=scheduler,
    trainloader=trainloader,
    validloader=testloader,
    validate_interval=None,
    epochs=10
)

In [8]:
trainer.run()

[Iter-102 Loss-0.976]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5273 - normalized_discounted_cumulative_gain@5(0.0): 0.6022 - mean_average_precision(0.0): 0.5671



[Iter-204 Loss-0.657]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.664 - normalized_discounted_cumulative_gain@5(0.0): 0.7286 - mean_average_precision(0.0): 0.6804



[Iter-306 Loss-0.487]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.686 - normalized_discounted_cumulative_gain@5(0.0): 0.7388 - mean_average_precision(0.0): 0.6925



[Iter-408 Loss-0.374]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.7179 - normalized_discounted_cumulative_gain@5(0.0): 0.7489 - mean_average_precision(0.0): 0.705



[Iter-510 Loss-0.293]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.7224 - normalized_discounted_cumulative_gain@5(0.0): 0.7576 - mean_average_precision(0.0): 0.7132



[Iter-612 Loss-0.182]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.7139 - normalized_discounted_cumulative_gain@5(0.0): 0.7453 - mean_average_precision(0.0): 0.7059



[Iter-714 Loss-0.098]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.7139 - normalized_discounted_cumulative_gain@5(0.0): 0.7453 - mean_average_precision(0.0): 0.7059



[Iter-816 Loss-0.094]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.7139 - normalized_discounted_cumulative_gain@5(0.0): 0.7453 - mean_average_precision(0.0): 0.7059



[Iter-918 Loss-0.103]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.7139 - normalized_discounted_cumulative_gain@5(0.0): 0.7453 - mean_average_precision(0.0): 0.7059



[Iter-1020 Loss-0.103]:
  Validation: normalized_discounted_cumulative_gain@3(0.0): 0.7139 - normalized_discounted_cumulative_gain@5(0.0): 0.7453 - mean_average_precision(0.0): 0.7059

Cost time: 10848.315711736679s
